In [1]:
from project_header import BearingNet

import numpy as np
import matplotlib.pyplot as plt
import os

from torch import nn, optim

## Read model

In [2]:
# ------- Read facenet -------
import torch

encode_size = 40
model = BearingNet(encode_size=encode_size)
model_path = './checkpoint/model_30_0.31.pth'
params = torch.load(model_path)
model.load_state_dict(params)
model.eval()

# -------- Read random forest -------
from joblib import load
RF = load('./checkpoint/RF.pth')

# ------- Read pattern vector -------
PatternVector = load('./data/Pattern_Vector/PatternVector.pth')

----------------------------------
| The model use Cuda:0 to train. |
---------------------------------- 



## Get encoding vector

In [4]:
from project_header import get_dict

test_dict = get_dict(path = './data/Image/test')
class_lst = ['Normal', 'Inner_break', 'Outer_break', 'Ball']
label_lst = []
vec_lst = torch.zeros(1, encode_size)
count = 0

with torch.no_grad():
    for cla in class_lst:
        length = test_dict[cla].shape[0]
        label_lst += [count for _ in range(length)]
        vec = model(test_dict[cla])
        vec_lst = torch.cat([vec_lst, vec], dim = 0)
        count += 1

vec_lst_torch = vec_lst[1:]
vec_lst_np = vec_lst_torch.numpy()
label_lst = np.array(label_lst)

print(vec_lst_np.shape, label_lst.shape)

(476, 40) (476,)


In [5]:
np.save('./data/Vector/x_test.npy', vec_lst)
np.save('./data/Vector/y_test.npy', label_lst)

## Facenet test

In [5]:
pred = np.zeros(label_lst.shape[0])
thrs = 0.0802
for i, vec in enumerate(vec_lst_torch):
    count = 0
    for key in PatternVector.keys():
        dist = torch.sqrt(torch.sum((vec - PatternVector[key]) ** 2).mean())
        # print(dist)
        if dist < thrs:
            count += 1
            break
    
    if count == 0:
        pred[i] = 3
    else:
        pred[i] = RF.predict(vec_lst_np[i].reshape(1, encode_size))[0]
    
    # if i == 10:
    #     break

In [7]:
from sklearn.metrics import classification_report
print(classification_report(label_lst, pred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97       118
           1       0.96      0.53      0.68       120
           2       0.99      0.97      0.98       119
           3       0.64      0.97      0.77       119

    accuracy                           0.85       476
   macro avg       0.90      0.85      0.85       476
weighted avg       0.90      0.85      0.85       476

